In [101]:
import pandas as pd
import numpy as np
import os

In [102]:
import os

def count_files_in_folder(folder_path):
    file_count = 0

    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            file_count += 1
    
    return file_count

print(f"Files in folder: {count_files_in_folder("data/")}.")

Files in folder: 349.


# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF

(Kolumny: OD M DO AB. Wiersze: Od 3 do 18 włącznie)
(wiersze 30 do 93 włącznie)
Wiersze 255 – 275 włącznie
Wiersze 279 – 287 włącznie (dane w %)
Wiersze 290 – 293 włącznie

In [103]:
folder_path = 'dataa/'

In [104]:
if os.path.exists(folder_path):
    print("Folder exists. Proceeding with file processing.")
else:
    print("Folder does not exist. Check the path.")

Folder exists. Proceeding with file processing.


In [105]:
# TEST, przejście po każdym pliku i wyciągnięcie z 'Info' nazwa firmy i sektora

In [106]:
def test_process_excel_files(folder_path):
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                with pd.ExcelFile(file_path) as xls:
                    company_name = pd.read_excel(xls, 'Info', usecols="B", skiprows=1, nrows=1).values[0][0]
                    sector = pd.read_excel(xls, 'Info', usecols="E", skiprows=19, nrows=1).values[0][0]
                
                result = {
                    'filename': filename,
                    'Company Name': company_name,
                    'Sector': sector
                }
                results.append(result)
                
                print(f"Dane z {filename} zostały przetworzone.")
            except Exception as e:
                print(f"Błąd przy przetwarzaniu pliku {filename}: {e}")

    results_df = pd.DataFrame(results)
    return results_df

In [107]:
test_process_excel_files(folder_path)

Dane z ZYWIEC.xlsx zostały przetworzone.


,filename,Company Name,Sector
0,ZYWIEC.xlsx,Grupa Żywiec SA,napoje


# Przygotowanie DF z danymi (kolumny)

In [108]:
def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    row_ranges = [
        range(2, 19),  # Rows 3 to 18 inclusive, zero-indexed
        range(29, 94),  # Rows 30 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 288),  # Rows 279 to 287 inclusive
        range(289, 294)  # Rows 290 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(file_path, sheet_name='QS', usecols=[2], header=None)

                extracted_data = []
                for row_range in row_ranges:
                    range_data = data.iloc[row_range].squeeze().tolist()
                    concatenated_data = ', '.join(map(str, range_data))
                    extracted_data.append(concatenated_data)

                new_row = {}
                for item in extracted_data:
                    for entry in item.split(', '):
                        new_row[entry.strip()] = None  # dummy value

                new_df = pd.DataFrame([new_row])
                new_df['filename'] = filename

                all_data.append(new_df)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.concat(all_data, ignore_index=True)
    return results_df

df_results = process_excel_data(folder_path)
df_results

,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,Cash flow from financing activities,...,Leverage (EM),Asset utilization (AU),Load gross profit,Load operating profit,EBITDA margin,EBITDA,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total),filename
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,ZYWIEC.xlsx


In [110]:
import os
import pandas as pd
import numpy as np

def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    # Zakresy wierszy, które chcemy wyciągnąć (przesunięte o jeden w dół, ponieważ indeksowanie jest od zera)
    row_ranges = [
        range(2, 19),  # Rows 3 to 18 inclusive, zero-indexed
        range(29, 94),  # Rows 30 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 288),  # Rows 279 to 287 inclusive
        range(289, 294)  # Rows 290 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                # Wczytujemy cały arkusz, aby mieć dostęp do kolumn C do DA
                data = pd.read_excel(file_path, sheet_name='QS', header=None)
                
                # Przechodzimy przez wszystkie zakresy wierszy i wyciągamy dane
                for row_range in row_ranges:
                    # Pobieramy wartości z kolumny C jako nagłówki
                    headers = data.iloc[list(row_range), 2].tolist()
                    
                    # Pobieramy dane z kolumn D do DA dla bieżącego zakresu wierszy
                    range_data = data.iloc[list(row_range), 3:30]
                    
                    # Zamiana zer na NaN
                    range_data = range_data.replace(0, np.nan)
                    
                    # Transponowanie danych, aby daty były w pierwszej kolumnie
                    transposed_data = range_data.transpose()
                    transposed_data.columns = headers
                    transposed_data['filename'] = filename
                    transposed_data.insert(0, 'date', data.iloc[2, 3:30].tolist())

                    all_data.append(transposed_data)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.concat(all_data, ignore_index=True)
    return results_df

df_results = process_excel_data(folder_path)
df_results


C:\Users\ksobc\AppData\Local\Temp\ipykernel_15724\1622711448.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  range_data = range_data.replace(0, np.nan)


,date,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,...,Rate debt security,Leverage (EM),Asset utilization (AU),Load gross profit,Load operating profit,EBITDA margin,EBITDA,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total)
0,1998-03-31,1998-03-31,1998-01-01,126003,11171,5812,NaN,11442,34508,-17981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-06-30,1998-06-30,1998-04-01,142773,50830,33203,NaN,12009,33829,-12482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998-09-30,1998-09-30,1998-07-01,222098,61141,41145,NaN,13995,52800,-12265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998-12-31,1998-12-31,1998-10-01,154110,27972,18590,NaN,12549,45540,-1632209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1999-03-31,1999-03-31,1999-01-01,168924,31190,18760,NaN,12387,95255,-138856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2003-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,89223.0,NaN,NaN,NaN
131,2003-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3554.0,NaN,NaN,NaN
132,2004-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60420.0,NaN,NaN,NaN
133,2004-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,81373.0,NaN,150000.0,150000.0


In [112]:
import os
import pandas as pd
import numpy as np

def process_excel_data(base_folder):
    folder_path = os.path.join(base_folder)
    all_data = []

    # Zakresy wierszy, które chcemy wyciągnąć (przesunięte o jeden w dół, ponieważ indeksowanie jest od zera)
    row_ranges = [
        range(2, 19),  # Rows 3 to 18 inclusive, zero-indexed
        range(30, 94),  # Rows 30 to 93 inclusive
        range(254, 276),  # Rows 255 to 275 inclusive
        range(278, 288),  # Rows 279 to 287 inclusive
        range(289, 294)  # Rows 290 to 293 inclusive
    ]

    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(folder_path, filename)
            try:
                data = pd.read_excel(file_path, sheet_name='QS', header=None)
                
                for row_range in row_ranges:
                    headers = data.iloc[list(row_range), 2].tolist()
                    
                    range_data = data.iloc[list(row_range), 3:30]
                    
                    range_data = range_data.replace(0, np.nan)
                    
                    # Transponowanie danych, aby daty były w pierwszej kolumnie
                    transposed_data = range_data.transpose()
                    transposed_data.columns = headers
                    
                    # Dodanie kolumny z nazwą pliku
                    transposed_data.insert(0, 'filename', filename)

                    all_data.append(transposed_data)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    results_df = pd.concat(all_data, ignore_index=True)
    return results_df

df_results = process_excel_data(folder_path)
df_results

C:\Users\ksobc\AppData\Local\Temp\ipykernel_15724\943373625.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  range_data = range_data.replace(0, np.nan)


,filename,End of period,Start of period,Revenues from sales,Operating profit/loss,Net profit/loss attributable to equity holders of the parent,Total comprehensive income attributable to equity holders,Depreciation,Cash flow from operating activities,Cash flow from investing activities,...,Rate debt security,Leverage (EM),Asset utilization (AU),Load gross profit,Load operating profit,EBITDA margin,EBITDA,Current financial liabilities,Non-current financial liabilities,Financial liabilities (total)
0,ZYWIEC.xlsx,1998-03-31,1998-01-01,126003,11171,5812,NaN,11442,34508,-17981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYWIEC.xlsx,1998-06-30,1998-04-01,142773,50830,33203,NaN,12009,33829,-12482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYWIEC.xlsx,1998-09-30,1998-07-01,222098,61141,41145,NaN,13995,52800,-12265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYWIEC.xlsx,1998-12-31,1998-10-01,154110,27972,18590,NaN,12549,45540,-1632209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYWIEC.xlsx,1999-03-31,1999-01-01,168924,31190,18760,NaN,12387,95255,-138856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,ZYWIEC.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,89223.0,NaN,NaN,NaN
131,ZYWIEC.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3554.0,NaN,NaN,NaN
132,ZYWIEC.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60420.0,NaN,NaN,NaN
133,ZYWIEC.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,81373.0,NaN,150000.0,150000.0


In [118]:
df_results.columns.tolist()

['filename',
 'End of period',
 'Start of period',
 'Revenues from sales',
 'Operating profit/loss',
 'Net profit/loss attributable to equity holders of the parent',
 'Total comprehensive income attributable to equity holders',
 'Depreciation',
 'Cash flow from operating activities',
 'Cash flow from investing activities',
 'Cash flow from financing activities',
 'Aktywa',
 'Non-current assets',
 'Current assets',
 'Equity shareholders of the parent',
 'Non-current liabilities',
 'Current liabilities',
 "Number of shares ('000)",
 'ASSETS',
 'Property, plant and equipment',
 'Exploration for and evaluation of mineral resources',
 'Intangible assets',
 'Goodwill',
 'Investment property',
 'Right-of-use assets',
 'Investment in affiliates',
 'Non-current financial assets',
 'Non-current loans and receivables',
 'Deferred income tax',
 'Non-current deferred charges and accruals',
 'Non-current derivative instruments',
 'Other non-current assets',
 'Inventories',
 'Current intangible asset

In [120]:
df_results = df_results.drop(columns='Start of period')

df = df.rename(columns={'Aktywa': 'sum_of_assets'})

In [122]:
# missing wiersz 45 Aktywa Obrotowe
# missing wiersz 62 
# missing wiersz 71
# missing wiersz 82

# pozmieniać nazwy niektórych kolumn z sumami
# pozmieniać nazwy kolumn na snakecase + małe litery
# drop kolumna 'nan


In [124]:
final_df = pd.DataFrame()

# # Lista kolumn do skopiowania
columns_to_copy = [
'Current financial liabilities', #291
'Non-current financial liabilities', #292
'Financial liabilities (total)' #293
'Return on equity (ROE)', #279
'Return on assets (ROA)', #280
'Leverage (EM)', #281
'Net profit margin', #282
'Asset utilization (AU)' #283
'Gross profit margin on sales', #255
'Operating profit margin', #256
'Gross profit margin', #257
'Net profit margin', #258
'Working capital ratio', #261
'Current ratio', #262
'Quick ratio', #263
'Cash ratio', #264
'Receivables turnover', #265 
'Inventory turnover', #266
'The operating cycle', #267
'Rotation commitments', #268
'Cash conversion cycle', #269
'Rotation assets', #270
'Rotation of assets', #271
'Assets ratio', #272
'Debt ratio', #273
'Debt service ratio', #274
'Rate debt security', #275
'ASSETS', #30 (sum-of-assets)
'Non-current assets', #31
'Property, plant and equipment', #32
'Exploration for and evaluation of mineral resources', #33
'Intangible assets', #34
'Goodwill',  #35
'Investment property', #36
'Right-of-use assets', #37
'Investment in affiliates', #38
'Non-current financial assets', #39
'Non-current loans and receivables', #40
'Deferred income tax', #41
'Non-current deferred charges and accruals', #42
'Non-current derivative instruments', #43
'Other non-current assets', #44
'Current assets', #45 
'Inventories', #46
'Current intangible assets', #47
'Biological assets', #48
'Trade receivables', #49
'Loans and other receivables', #50
'Financial assets', #51
'Cash and cash equivalents', #52
'Accruals', #53
'Assets from current tax', #54
'Derivative instruments', #55
'Other assets', #56
'Assets held for sale and discontinuing operations', #57
'Called up capital', #58
'Own shares', #59
'EQUITY & LIABILITIES', #61
'Equity shareholders of the parent', #62
'Share capital', #63
'Called up share capital', #64
'Treasury shares', #65
'Supplementary capital', #66
'Valuation and exchange differences', #67
'Other capitals', #68
'Retained earnings / accumulated losses', #69
'Non-controlling interests', #70
'Non-current liabilities', #71
'Non-current liabilities from derivatives', #72
'Non-current loans and borrowings', #73
'Non-current liabilities from bonds', #74
'Non-current liabilities from finance leases', #75
'Non-current trade payables', #76
'Long-term provision for employee benefits', #77
'Deferred tax liabilities', #78
'Non-current provision', #79
'Other non-current liabilities', #80
'Non-current accruals (liability)', #81
'Current liabilities', #82
'Liabilities from derivatives', #83
'Financial liabilities (loans and borrowings)', #84
'Bond liabilities', #85
'Liabilities from finance leases', #86
'Trade payables', #87
'Employee benefits', #88
'Current tax liabilities', #89
'Provisions', #90
'Other liabilities',  #91
'Accruals (liability)', #92
'Liabilities related to assets held for sale and discontinued operations', #93

'End of period', #3 (second column)
'Revenues from sales', #5
'Operating profit/loss', #6
'Net profit/loss attributable to equity holders of the parent', #7
'Total comprehensive income attributable to equity holders', #8
'Depreciation', #9
'Cash flow from operating activities', #10
'Cash flow from investing activities', #11
'Cash flow from financing activities', #12
'Non-current assets', #14
'Current assets', #15
'Equity shareholders of the parent', #16
'Non-current liabilities', #17
'Current liabilities', #18
'Number of shares ('000)', #19
'Earnings per share', #20
'Total comprehensive income per share', #21
'Book value per share' #22
]
# te marże mogą być zbędne i powodować niepotrzebny szum (ale nie wiem) 
# może drugi dataset bez marży
# # Dodawanie wybranych kolumn do pustego DataFrame
# df_empty = df_source[kolumny_do_skopiowania]